<a href="https://colab.research.google.com/github/pyagoubi/Stuff/blob/main/stockprediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
%%capture
!pip install bt

In [29]:
%matplotlib inline
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from torch.utils.data import dataloader
import bt as bt
import os, sys, itertools, urllib, io
import datetime as dt
import pandas as pd
import pandas_datareader as dr
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset

In [19]:
tech_daily_raw = pd.read_csv('/content/drive/MyDrive/stock predict/technical/1D_technical.csv')
features = ['open', 'high', 'low', 'close', 'rsi', 'adx', 'cci', 'ema', 'stoch', 'trend_macd', 'momentum_stoch', 'volatility_atr']
tech_daily = tech_daily_raw[features]

# init deterministic seed
seed_value = 1234
np.random.seed(seed_value) # set numpy seed
torch.manual_seed(seed_value); # set pytorch seed CPU

# set cpu or gpu enabled device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type

# init deterministic GPU seed
torch.cuda.manual_seed(seed_value)

# log type of device enabled
now = dt.datetime.utcnow().strftime("%Y%m%d-%H:%M:%S")
print('[LOG {}] notebook with \'{}\' computation enabled'.format(str(now), str(device)))





[LOG 20230516-13:09:35] notebook with 'cuda' computation enabled


In [20]:
tech_daily = tech_daily.copy()
tech_daily['return'] = np.log(tech_daily['close']) - np.log(tech_daily['close'].shift(1))

In [21]:
features = features + ['return']

In [25]:
scaler = MinMaxScaler()
tech_daily_scaled = pd.DataFrame(scaler.fit_transform(tech_daily), columns = features)

In [7]:
split_fraction = 0.9
split_row = int(tech_daily.shape[0] * split_fraction)
train_stock_data_return = tech_daily.iloc[:split_row]
valid_stock_data_return = tech_daily.iloc[split_row:]

In [53]:
time_steps = 4 # number of predictor timesteps
horizon = 1 # number of timesteps to be predicted
sequence_length = time_steps + horizon # determine sequence length

In [36]:
def preprocess_data(df, sequence_length):
    # Normalize the data
    #normalized_data = (df - df.mean()) / df.std()

    # Convert the DataFrame to a numpy array
    #data_array = normalized_data.values

    data_array = df.values

    # Split the data into sequences
    sequences = []
    for i in range(len(data_array) - sequence_length):
        sequence = data_array[i : i + sequence_length + 1]
        sequences.append(sequence)

    # Convert sequences to PyTorch tensors
    tensor_sequences = [torch.Tensor(seq) for seq in sequences]

    # Stack the tensor sequences
    stacked_sequences = torch.stack(tensor_sequences)

    return stacked_sequences


class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data) - 1

    def __getitem__(self, idx):
        x = torch.Tensor(self.data[idx, :-1])  # Input features (all columns except the last one)
        y = torch.Tensor([self.data[idx + 1, -1]])  # Target variable ('return' column for the next time step)
        return x, y

In [ ]:
train_stock_data_return = preprocess_data(df, sequence_length) 
valid_stock_data_return

In [ ]:
dataset = MyDataset(preprocessed_data)